## Imports

In [4]:
from datasets import load_dataset, ClassLabel

In [5]:
VERSION = "v1"
!ls -lah datasets | grep {VERSION} | grep qnli
# !ls -lah datasets | grep {VERSION} | grep imdb
# !ls -lah datasets | grep {VERSION}

drwxr-xr-x  3 augustas Domain Users  25K Jul 21 22:46 qnli_vicuna_ppo_training_raw_v1
-rw-r--r--  1 augustas Domain Users  19M Jul 21 23:18 qnli_vicuna_ppo_training_v1.parquet
drwxr-xr-x  3 augustas Domain Users  33K Jul 21 22:45 qnli_vicuna_train_raw_v1
-rw-r--r--  1 augustas Domain Users 2.0M Jul 21 23:02 qnli_vicuna_train_v1.parquet
-rw-r--r--  1 augustas Domain Users 1.1M Jul 21 23:26 qnli_vicuna_validation_v1.parquet


In [17]:
# parquet_data_files = {
#     "train": f"datasets/burns_datasets_VINC_train_{VERSION}.parquet",
#     "validation": f"datasets/burns_datasets_VINC_validation_{VERSION}.parquet",
# }
# parquet_data_files = {
#     "train": f"datasets/burns_datasets_VINC_imdb_train_{VERSION}.parquet",
#     "validation": f"datasets/burns_datasets_VINC_imdb_validation_{VERSION}.parquet",
# }
parquet_data_files = {
    "train": f"datasets/qnli_vicuna_train_{VERSION}.parquet",
    "validation": f"datasets/qnli_vicuna_validation_{VERSION}.parquet",
}

# parquet_data_files = {
#     "train": f"datasets/burns_datasets_VINC_ppo_training_{VERSION}.parquet",
# }
# parquet_data_files = {
#     "train": f"datasets/burns_datasets_VINC_imdb_ppo_training_{VERSION}.parquet",
# }
# parquet_data_files = {
#     "train": f"datasets/qnli_vicuna_ppo_training_{VERSION}.parquet",
# }

my_dataset = load_dataset("parquet", data_files=parquet_data_files)
my_dataset

Found cached dataset parquet (/admin/home-augustas/.cache/huggingface/datasets/parquet/default-0422c8c7bd6c8c48/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'original_dataset', 'text'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['label', 'original_dataset', 'text'],
        num_rows: 5463
    })
})

In [18]:
if len(my_dataset.keys()) == 2:
    for split in my_dataset:
        print(split)
        
        features = my_dataset[split].features
        names = ["neg", "pos"]
        if "qnli" in parquet_data_files["train"]: names = ["entailment", "not_entailment"]
        
        features["label"] = ClassLabel(names=names)
        my_dataset[split] = my_dataset[split].map(features=features, load_from_cache_file=False)
        print(my_dataset[split].features["label"])
else:
    print("PPO dataset - only one split")

train


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

ClassLabel(names=['entailment', 'not_entailment'], id=None)
validation


Map:   0%|          | 0/5463 [00:00<?, ? examples/s]

ClassLabel(names=['entailment', 'not_entailment'], id=None)


In [20]:
# if len(my_dataset.keys()) == 2:
#     print("Train/validation dataset")
#     # my_dataset.push_to_hub(f"AugustasM/burns-datasets-VINC-{VERSION}", private=True)
#     # my_dataset.push_to_hub(f"AugustasM/burns-datasets-VINC-imdb-{VERSION}", private=True)
#     my_dataset.push_to_hub(f"AugustasM/qnli-vicuna-{VERSION}", private=True)
# else:
#     print("PPO dataset")
#     # my_dataset.push_to_hub(f"AugustasM/burns-datasets-VINC-ppo-training-{VERSION}", private=True)
#     # my_dataset.push_to_hub(f"AugustasM/burns-datasets-VINC-imdb-ppo-training-{VERSION}", private=True)
#     my_dataset.push_to_hub(f"AugustasM/qnli-vicuna-ppo-training-{VERSION}", private=True)

## Test

In [21]:
# tmp = load_dataset(f"AugustasM/burns-datasets-VINC-{VERSION}")
# tmp = load_dataset(f"AugustasM/burns-datasets-VINC-imdb-{VERSION}")
tmp = load_dataset(f"AugustasM/qnli-vicuna-{VERSION}")

tmp["train"].features["label"], tmp["validation"].features["label"]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5463 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /admin/home-augustas/.cache/huggingface/datasets/AugustasM___parquet/AugustasM--qnli-vicuna-v1-aba457150f5153b4/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

(ClassLabel(names=['entailment', 'not_entailment'], id=None),
 ClassLabel(names=['entailment', 'not_entailment'], id=None))

In [22]:
tmp

DatasetDict({
    train: Dataset({
        features: ['label', 'original_dataset', 'text'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['label', 'original_dataset', 'text'],
        num_rows: 5463
    })
})

In [23]:
# import datasets
# datasets.disable_progress_bar()
# datasets.logging.set_verbosity_error()


# for i in range(1, 6):
#     tmp = load_dataset(f"AugustasM/burns-datasets-VINC-imdb-v1{i}")
#     print(tmp["train"][0]["text"][:100])
#     print(tmp["validation"][0]["text"][:100])
#     print("-" * 80)

In [24]:
# tmp = load_dataset(f"AugustasM/burns-datasets-VINC-ppo-training-{VERSION}")
# tmp = load_dataset(f"AugustasM/burns-datasets-VINC-imdb-ppo-training-{VERSION}")
tmp = load_dataset(f"AugustasM/qnli-vicuna-ppo-training-{VERSION}")
tmp

Found cached dataset parquet (/admin/home-augustas/.cache/huggingface/datasets/AugustasM___parquet/AugustasM--qnli-vicuna-ppo-training-v1-e9526ea8884b4bb3/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['original_dataset', 'prompt', 'best_response'],
        num_rows: 94743
    })
})